<a href="https://colab.research.google.com/github/moyocoyani/dataviz_eng_tips/blob/main/ColorMappers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use color mappers in bokeh?

*Author: Moyocoyani Molina-Espíritu*

In this notebook we will put examples about color mapping in bokeh.

This is super useful when you want to apply some color palettes straight to your data.

The way that bokeh works is similar to other libraries, the main aspect here is that you need to provide the palette in any of the bokeh mappers:

- CategoricalColorMapper

- LinearColorMapper

- LogColorMapper

```
mapper = ColorMapper(palette = A_LIST_WITH_COLORS_GOES_HERE,
                     *arguments,
                     **keywordarguments)
```

Then you will need to add any of those mappers into a dictionary with the Keys `field` and `transform`.

```
color = {"field": NAME_OF_COLUMN_AS_STRING_GOES_HERE,
         "transform": mapper }
```

The first one, `field` is for the name of the column that contains the data that you want to map, and the `transform` key will holds your mapper.
In the rest of the notebook I show you how to config those mappers with the bare minimum and how to implement them in bokeh; we will use bokeh palettes but you can apply any sequence of colors.


## Prepare the data

1. First let's import some bokeh modules and numpy. The last one only for creating our dummy data.

In [ ]:
import numpy as np         #data creation
import bokeh.models as bkm #most bokeh modules are contained here

from bokeh.plotting import figure                  #figure is the easiest way to create plots with bokeh
from bokeh.palettes import Category10_4, Viridis256 #Bokeh has its own palettes
from bokeh.layouts import column
from bokeh.io import output_notebook, show         #We will show everything in the notebook

2. Let's create some dummy variables:
- A categorical variable so we can map categorical data.
- 2 numerical variables in linear scale.
- 1 numerical variable in logarithmic scale.

In [ ]:
gps = 20*["Group A"] + \
      10*["Group B"] + \
     100*["Group C"] + \
      70*["Group D"]
x = np.linspace(0,20,200)
y = 10 + np.cos(np.linspace(0,20,200)) + np.random.rand(200)
size = 3* np.logspace(.2,1,200)

3. Now we can create a dictionary with those variables

In [ ]:
src = dict( gps = gps,
            x = x,
            y = y,
            size = size
          )

4. With this data we can create our `ColumnDataSource` which is a bokeh model that handles the data structure for bokeh plots. See the [documentation here](https://docs.bokeh.org/en/latest/docs/reference/models/sources.html#bokeh.models.ColumnDataSource)

In [ ]:
source = bkm.ColumnDataSource(src)

5. Plot our dummy data.

## Plot without color mapping.

In [ ]:
p = figure(width=640,
           height = 480)

p.scatter(x="x",
          y="y",
          size="size",
          fill_alpha=.7,
          source=source)

output_notebook()
show(p)

## Plot with categorical color mapping.

In [ ]:
#Create a mapper. Cateforical requires the categories as factors
mapper = bkm.CategoricalColorMapper(factors=["Group A",
                                            "Group B",
                                            "Group C",
                                            "Group D"],
                                    palette = Category10_4)

#Then creates the dictionary with the mapper and the field
color = dict(field = "gps", transform = mapper)

p = figure(width=640,
           height = 480)

p.scatter(x="x",
y="y",
size = "size",
fill_color = color,
line_color = color,
fill_alpha = 0.7,
source=source)

output_notebook()
show(p)

## Plot with linear color mapping.

In [ ]:
#Cretaes the mapper
mapper = bkm.LinearColorMapper(palette = Viridis256)

#Then creates the dictionary with the mapper and the field
color = dict(field = "y",
transform = mapper)

p = figure(width=640,
           height = 480)
p.scatter(x="x",
y="y",
size = "size",
fill_color = color,
line_color = color,
fill_alpha = 0.7,
source=source)


output_notebook()
show(p)

## Plot with log-color mapping.

In [ ]:
#Creates the mapper
mapper = bkm.LogColorMapper(palette = Viridis256)

#Then creates the dictionary with the mapper and the field
color = dict(field = "size",
transform = mapper)

p = figure(width=640,
           height = 480)
p.scatter(x="x",
y="y",
size = "size",
fill_color = color,
line_color = color,
fill_alpha = 0.7,
source=source)

output_notebook()
show(p)

# Extra tips. Giving some format.

This is how we obtain the charts of the publication:

In [ ]:
def plot_format(p, mapper=None, color_title=None):
    grids_color = "#a3a3a3"
    p.outline_line_width = 0
    text_font_size = "1.2rem"
    label_size  = "0.85rem"

    #give format to 2 axes and grids (x and y)
    for i in range(2):
        p.axis[i].axis_line_color = None
        p.axis[i].minor_tick_line_width = 0
        p.axis[i].major_tick_line_width = 0
        p.axis[i].major_label_text_color = grids_color
        p.axis[i].major_label_text_font_size = text_font_size


        p.grid[i].grid_line_color = grids_color
        p.grid[i].grid_line_dash = "dashed"


    #Adding titles
    x_title = bkm.Title(text="X axis title",
                        text_font_size=text_font_size,
                        text_font_style="normal")
    y_title = bkm.Title(text="Y axis title",
                        text_font_size=text_font_size,
                        align = "right",
                        text_font_style="normal"
                        )

    p.add_layout(x_title,"below")
    p.add_layout(y_title,"left")

    #Adding color if any
    if mapper is not None:
      #Adding color
      color_bar = bkm.ColorBar(color_mapper = mapper,
                              height=10,
                              width = 300,
                              location="bottom_left",
                              title=color_title,
                              title_text_font_style="normal",
                               title_text_font_size=text_font_size,
                               major_label_text_font_size=label_size)
      p.add_layout(color_bar, 'below')
    return p

## Plot without color mapping.

In [ ]:
p = figure(width=640,
           height = 480)

p.scatter(x="x",
          y="y",
          size="size",
          fill_alpha=.7,
          source=source)

title = bkm.Div(text="<div style='font-size: 1.75rem; font-weight:bold;margin-bottom:15px;'>Plot without color mapping.</div>",
                  )

p = column([title,plot_format(p)])
output_notebook()
show(p)

## Plot with categorical color mapping.

In [ ]:
mapper = bkm.CategoricalColorMapper(factors=["Group A",
"Group B",
"Group C",
"Group D"],
palette = Category10_4)

color = dict(field = "gps", transform = mapper)

p = figure(width=640,
           height = 480)

p.scatter(x="x",
y="y",
size = "size",
fill_color = color,
line_color = color,
fill_alpha = 0.7,
source=source)


#Adding a title
title = bkm.Div(text="<div style='font-size: 1.75rem; font-weight:bold;margin-bottom:15px;'>Plot with categorical mapping.</div>",
                  )

p = column([title,plot_format(p,mapper,"Groups")])
output_notebook()
show(p)

## Plot with linear color mapping.

In [ ]:
mapper = bkm.LinearColorMapper(
palette = Viridis256)

color = dict(field = "y",
transform = mapper)

p = figure(width=640,
           height = 480)

p.scatter(x="x",
y="y",
size = "size",
fill_color = color,
line_color = color,
fill_alpha = 0.7,
source=source)


#Adding a title
title = bkm.Div(text="<div style='font-size: 1.75rem; font-weight:bold;margin-bottom:15px;'>Plot with linear color mapping.</div>",
                  )

p = column([title,plot_format(p,mapper, "Y values")])
output_notebook()
show(p)

## Plot with log-color mapping.

In [ ]:
mapper = bkm.LogColorMapper(
palette = Viridis256)

color = dict(field = "size",
transform = mapper)


p = figure(width=640,
           height = 480)

p.scatter(x="x",
y="y",
size = "size",
fill_color = color,
line_color = color,
fill_alpha = 0.7,
source=source)

#Adding a title
title = bkm.Div(text="<div style='font-size: 1.75rem; font-weight:bold;margin-bottom:15px;'>Plot with logarithmic color mapping.</div>",
                  )

p = column([title,plot_format(p,mapper,"Size")])
output_notebook()
show(p)